Ferramentas de NLP usando o dataset de tweets


In [1]:
from google.colab import drive
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

Baixando o stopwords de todas as línguas.

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Dando permissão para o colab poder acessar os arquivos do meu drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Lendo o DataSet a partir do drive

In [ ]:
df = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/train.csv')

df.head()

,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,Observação,Id
0,Mon Jan 09 15:27:43 +0000 2017,Dois são detidos ao tentar jogar celulares e d...,NaN,NaN,NaN,Michele #beta #sdv,michelexmbeta,0,Positivo,NaN,6272
1,Sun Jan 08 02:14:34 +0000 2017,me matan esas minas q cambian 554 veces su fot...,NaN,NaN,Núñez - C.A.B.A.,Gaby Messina,gabymessina36,0,Neutro,NaN,1644
2,Sat Feb 11 09:49:11 +0000 2017,Líderes de motim em presídio de Minas Gerais s...,NaN,NaN,"Hollywood, CA",Wendie Rower,Wendie_Rower,0,Positivo,NaN,7956
3,Thu Jan 05 14:43:03 +0000 2017,#Mídia: Press Release from Business Wire : Di...,NaN,NaN,SP,Marcello Binder,binderbr,0,Neutro,NaN,85
4,Wed Feb 08 22:52:10 +0000 2017,Vacinação contra febre amarela é intensificada...,NaN,NaN,NaN,fodido,eufodeu,0,Positivo,NaN,6006


Pegando uma amostra do DataSet

In [ ]:
data = df.Text

sentence = data[700]
print(sentence)

RT @vemprarua: Arroz de terceira!     BB avisa Justiça que governo de MG não tem recursos em conta judicial https://t.co/SZtiWxZtKt


Fazendo a Tokenização

reduce_len. Diminui as palavras alongadas

strip_handles Desconsidera palavras junto com o @

In [5]:
from nltk.tokenize.casual import casual_tokenize

tokens = casual_tokenize(sentence, reduce_len=True, strip_handles=True)

print(tokens)

NameError: ignored

Tokenização por TFID

In [ ]:
corpus = [
        'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?',
]

teste = []

teste.append("Arthurn G")
teste.append("AR")

vectorizer = TfidfVectorizer(min_df=1)
tf_idf = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
teste

['Arthurn G', 'AR']

Criando a Lista de StopWords inicialmente para as três linguas

In [ ]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

len(stop_words)

696

Normalizando o vocabulario para letra minusculas


In [ ]:
normalized_tokkens = [x.lower() for x in tokens]
print(normalized_tokkens)

['rt', ':', 'arroz', 'de', 'terceira', '!', 'bb', 'avisa', 'justiça', 'que', 'governo', 'de', 'mg', 'não', 'tem', 'recursos', 'em', 'conta', 'judicial', 'https://t.co/sztiwxztkt']


Fazendo o STEM que transforma as palavras para raiz como por exemplo judicial = judici

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
' '.join(stemmer.stem(w).strip("'") for w in tokens)

'RT : arroz de terceira ! BB avisa justiça que governo de MG não tem recurso em conta judici https://t.co/sztiwxztkt'

Fazendo a Lemmatização que transforma gato, gata em gat por exemplo

In [ ]:
from nltk.stem import  WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize(sentence, pos='n')

'RT @vemprarua: Arroz de terceira!     BB avisa Justiça que governo de MG não tem recursos em conta judicial https://t.co/SZtiWxZtKt'

Função para obter o pos de forma automatica para após isso fazer a Lemmatização


In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.wordnet.wordnet.ADJ,
                "N": nltk.wordnet.wordnet.NOUN,
                "V": nltk.wordnet.wordnet.VERB,
                "R": nltk.wordnet.wordnet.ADV}

    return tag_dict.get(tag, nltk.wordnet.wordnet.NOUN)

t = get_wordnet_pos('good')
lemmatizer.lemmatize('better', pos=t)

'good'

Criando a Bag of Wods

In [ ]:
bags_of_words = []

for text in df.Text:
  bags_of_words.append(Counter(casual_tokenize(text)))

df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
print(df_bows.shape)
print(df_bows.head())

(6559, 12641)
   Dois  são  ...  https://t.co/1QtSEWtwAL  https://t.co/pxpixAgcPw
0     1    1  ...                        0                        0
1     0    0  ...                        0                        0
2     0    0  ...                        0                        0
3     0    0  ...                        0                        0
4     0    0  ...                        0                        0

[5 rows x 12641 columns]


NGrams

In [ ]:
list(ngrams(tokens,2))

[(('rt', '2'), ('2', 'helicópteros')),
 (('2', 'helicópteros'), ('helicópteros', 'cara')),
 (('helicópteros', 'cara'), ('cara', 'pau')),
 (('cara', 'pau'), ('pau', 'canalhice')),
 (('pau', 'canalhice'), ('canalhice', 'ainda')),
 (('canalhice', 'ainda'), ('ainda', 'ñ')),
 (('ainda', 'ñ'), ('ñ', 'maiores')),
 (('ñ', 'maiores'), ('maiores', 'q')),
 (('maiores', 'q'), ('q', 'calamidade')),
 (('q', 'calamidade'), ('calamidade', 'financeira')),
 (('calamidade', 'financeira'), ('financeira', 'vive')),
 (('financeira', 'vive'), ('vive', 'htt'))]

Remover Link

In [ ]:
sentence = re.sub(r'http\S+', '', sentence)

In [ ]:
sentence

'RT @vemprarua: Arroz de terceira!     BB avisa Justiça que governo de MG não tem recursos em conta judicial '

Remover Pontuação

In [ ]:
# tokens = casual_tokenize(sentence, reduce_len=True, strip_handles=True)
# tokens = "Arthur @# ☺️🐯🍈 6+5"
# tokens = casual_tokenize(tokens)
# puncs = set((',', '.', '--','-','!','?',':', ';', '``', "''", '(', ')', '[', ']','\u2026','"'))
# tokens = (x.lower() for x in tokens if x not in puncs)
tokens = [word for word in tokens if word.isalnum()]
tokens = Counter(tokens)
print(tokens)

Counter({'arthur': 1, '@': 1, '#': 1, '☺': 1, '️': 1, '🐯': 1, '🍈': 1, '6': 1, '+': 1, '5': 1})


Remover Stopwords

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')
tokens = [x for x in tokens if x not in stopwords]
tokens = Counter(tokens)
print(tokens)

Counter({'rt': 1, 'arroz': 1, 'terceira': 1, 'bb': 1, 'avisa': 1, 'justiça': 1, 'governo': 1, 'mg': 1, 'recursos': 1, 'conta': 1, 'judicial': 1})


In [ ]:
tokens

['rt',
 '2',
 'helicópteros',
 'cara',
 'pau',
 'canalhice',
 'ainda',
 'ñ',
 'maiores',
 'q',
 'calamidade',
 'financeira',
 'vive',
 'htt']

In [ ]:
list(ngrams(tokens,2))

[('rt', '2'),
 ('2', 'helicópteros'),
 ('helicópteros', 'cara'),
 ('cara', 'pau'),
 ('pau', 'canalhice'),
 ('canalhice', 'ainda'),
 ('ainda', 'ñ'),
 ('ñ', 'maiores'),
 ('maiores', 'q'),
 ('q', 'calamidade'),
 ('calamidade', 'financeira'),
 ('financeira', 'vive'),
 ('vive', 'htt')]